In [0]:
dbutils.fs.rm("abfss://bronze@adlsathlete.dfs.core.windows.net/strava/sub_activity/checkpoints/", recurse=True)


In [0]:

from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import col
class BronzeLoader:
    def __init__(self, input_path, checkpoint_path, format_file):
        self.input_path = input_path
        self.checkpoint_path = checkpoint_path
        self.format_file = format_file

    def read_autoloader(self, schema=None):
        df = (spark.readStream
              .format("cloudFiles")
              .option("cloudFiles.format", self.format_file)
              .option("cloudFiles.inferColumnTypes", "true")
              .option("cloudFiles.schemaEvolutionMode", "addNewColumns")
              .option("cloudFiles.schemaLocation", self.checkpoint_path)
              .load(self.input_path)
             )
        return df if not schema else df.selectExpr(*schema)

    def standardize_columns(self, df):
        return (df
                .withColumn("athlete_id", col("athlete.id"))
                .withColumn("ingestion_timestamp", current_timestamp())
               )
    
    def write_to_bronze(self, df, output_path):
        return (df.writeStream
              .format("delta")  # <-- aqui muda
              .option("checkpointLocation", self.checkpoint_path)
              .option("mergeSchema", "true")
              .outputMode("append")
              .trigger(availableNow=True)
              .start(output_path)
           )


In [0]:
bronze = BronzeLoader(
    input_path="abfss://raw@adlsathlete.dfs.core.windows.net/strava/sub_activity/",
    checkpoint_path="abfss://bronze@adlsathlete.dfs.core.windows.net/strava/sub_activity/checkpoints/",
    format_file="json"
    
)

df = bronze.read_autoloader()
df = bronze.standardize_columns(df)
df.display()

bronze.write_to_bronze(
    df,
    output_path="abfss://bronze@adlsathlete.dfs.core.windows.net/strava/sub_activity/"
)

In [0]:
%sql
CREATE TABLE IF NOT EXISTS `uc_athlete_data`.bronze.strava_sub_activity
USING DELTA 
COMMENT "Tabela com SUB atividades dos atletas no Strava"
LOCATION 'abfss://bronze@adlsathlete.dfs.core.windows.net/strava/sub_activity/'

In [0]:
%sql
select * from `uc_athlete_data`.bronze.strava_sub_activity